# 🤖 Деревья решений и ансамблевые методы 🤖

## 👨🏻‍💻 Практика 3 - Классификация пациентов на больных и здоровых при несбалансированных данных.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.utils import class_weight

import numpy as np

In [2]:
# убираем warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

### ✔ 1.Генерация данных: 
Создаем DataFrame с несбалансированными классами (больные/здоровые).

In [3]:
# Создание DataFrame с данными (пример)
data = {'symptom_1': [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0],
        'symptom_2': [0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0],
        'symptom_3': [1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1],
        'target': [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]}
df = pd.DataFrame(data)
df

,symptom_1,symptom_2,symptom_3,target
0,1,0,1,1
1,0,1,1,0
2,1,1,0,0
3,0,0,1,0
4,1,1,0,0
5,0,1,1,0
6,1,0,1,0
7,1,1,0,1
8,0,0,1,0
9,1,1,0,0


In [4]:
# Разделение на признаки и целевую переменную
X = df.drop('target', axis=1)
y = df['target']

### ✔ 2.  Исследование: 
Выводим количество объектов каждого класса, чтобы подтвердить несбалансированность.

In [5]:
print(f"Количество объектов каждого класса: {y.value_counts()}")

Количество объектов каждого класса: target
0    16
1     4
Name: count, dtype: int64


Видно, что данные несбалансированы: класс 1 значительно меньше

### ✔ 3. Разделение данных: 
Разделяем данные на обучающую и тестовую выборки.

In [6]:
# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = (
    train_test_split(X, y, test_size=0.2, random_state=42))

### ✔  4. Базовая модель: 
Обучаем DecisionTreeClassifier без учета несбалансированности. Вычисляем метрики качества, обращая внимание на низкий recall для класса меньшинства.
### ✔ 5. Взвешивание классов: 
Используем class_weight='balanced' для автоматического назначения весов классам. Оцениваем метрики, наблюдая, как улучшается recall для  класса меньшинства.
### ✔ 6.SMOTE: 
Применяем SMOTE для генерации синтетических данных класса меньшинства. Обучаем модель на сбалансированных данных. Снова оцениваем метрики.

In [7]:
# 4 Модель без учета несбалансированности

clf_base = DecisionTreeClassifier(random_state=42)
clf_base.fit(X_train, y_train)
y_pred_base = clf_base.predict(X_test)

print("\nБазовая модель:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_base):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_base):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_base):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_base):.4f}")
print(f"ROC-AUC: \
{roc_auc_score(y_test, clf_base.predict_proba(X_test)[:, 1]):.4f}")
# 5 Модель с взвешиванием классов
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',  # Выбираем балансировку
    classes=np.unique(y_train),  # Список уникальных классов, 
    y= y_train) # Целевые переменные

# Преобразуем результат в словарь
class_weight_dict = (
    {i: class_weights[i] for i in range(len(class_weights))}
)

# Теперь можно передать в модель
clf_balanced = DecisionTreeClassifier(
    random_state=42, 
    class_weight=class_weight_dict)

clf_balanced.fit(X_train, y_train)
y_pred_balanced = clf_balanced.predict(X_test)

print("\nМодель с взвешиванием классов:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_balanced):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_balanced):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_balanced):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_balanced):.4f}")
print(f"ROC-AUC: \
{roc_auc_score(y_test, clf_balanced.predict_proba(X_test)[:, 1]):.4f}")

# 6 Модель с SMOTE

smote = SMOTE(random_state=42, k_neighbors=2)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

clf_smote = DecisionTreeClassifier(random_state=42)
clf_smote.fit(X_train_smote, y_train_smote)
y_pred_smote = clf_smote.predict(X_test)
print("\nМодель с SMOTE:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_smote):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_smote):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_smote):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_smote):.4f}")
print(f"ROC-AUC: \
{roc_auc_score(y_test, clf_smote.predict_proba(X_test)[:, 1]):.4f}")


Базовая модель:
Accuracy: 0.7500
Precision: 0.0000
Recall: 0.0000
F1-score: 0.0000
ROC-AUC: 0.3333

Модель с взвешиванием классов:
Accuracy: 0.5000
Precision: 0.0000
Recall: 0.0000
F1-score: 0.0000
ROC-AUC: 0.3333

Модель с SMOTE:
Accuracy: 0.5000
Precision: 0.0000
Recall: 0.0000
F1-score: 0.0000
ROC-AUC: 0.3333


### ✔7. Вывод:

Важно сравнивать результаты разных подходов к борьбе с несбалансированностью. В данном примере, SMOTE и взвешивание классов могут привести к улучшению метрик для класса меньшинства, что важно для задачи медицинской диагностики, где ошибки в предсказании